# $U(1)$ Gauge Model using L2HMC in graph mode

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

from l2hmc_eager import gauge_dynamics_eager as gde

Using TensorFlow backend.


In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [47]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 8.,
    'num_samples': 10,
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 5e-5,
    'learning_rate_decay_steps': 500,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 5000,
    #'record_loss_every': 50,
    #'data_steps': 100,
    'save_steps': 1000,
    #'print_steps': 5,
    'logging_steps': 50,
    'clip_value': 10,
    'rand': False,
    'metric': 'l2',
}
tf.reset_default_graph()

## Create model

In [48]:
#del model

In [49]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [50]:
model = GaugeModel(params=params,
                   config=config,
                   sess=None,
                   conv_net=False,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True,
                   aux=False)
# 225.39 --> 419.52 w/ aux. fn
# 137.54 --> 228.63 w/ aux. fn

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/
log_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/
info_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/run_info/
figs_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/figures/
conv_net: False
hmc: False
aux: False
eps_trainable: True
time_size: 8
space_size: 8
link_type: U1
dim: 2
beta: 8.0
num_samples: 10
num_steps: 5
eps: 0.2
loss_scale: 0.1
loss_eps: 0.0001
learning_rate_init: 5e-05
learning_rate_decay_steps: 500
learning_rate_decay_rate: 0.98
train_steps: 5000
save_steps: 1000
logging_steps: 50
clip_value: 10
rand: False
metric: l2
samples_history_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/samples_history
--------------------------------------------------------------------------------

Building graph... (started at: Sat Jan 12 12:33:29 2019)
  Creating loss...

    took: 1087.8854410648346 seconds.
  Creating optimizer...

    took: 462.16167879104614 seconds.
  Cre

In [51]:
model.train(model.train_steps, kill_sess=False)

--------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS            LR      
--------------------------------------------------------------------------------------
Time to complete logging: 247.1 

     9/5000       -630.1       0.0001324      0.03227       0.1996         5e-05    
    19/5000        149.3       0.0001224      0.01847       0.1993         5e-05    
    29/5000        997.1       0.0001241     2.523e-10       0.199         5e-05    
    39/5000        1000        0.000119      1.404e-12      0.1988         5e-05    
    49/5000        898.8       0.0001697     3.145e-06      0.1987         5e-05    
Time to complete logging: 0.3986

    59/5000        522.8       0.0001193      0.05182       0.1987         5e-05    
    69/5000        1000        0.0001323     6.693e-13      0.1986         5e-05    
    79/5000        1000        0.0001281     8.653e-17      0.1985         5e

   879/5000       -689.2       0.0001107      0.1676         0.173        4.9e-05   
   889/5000        -1672       0.0001123      0.3897        0.1727        4.9e-05   
   899/5000        -2408       0.0001125      0.4785        0.1725        4.9e-05   
Time to complete logging: 0.2652

   909/5000        -1948       0.0001121      0.4681        0.1723        4.9e-05   
   919/5000        -1780       0.0001234      0.3743         0.172        4.9e-05   
   929/5000       -764.4       0.0001129      0.1193        0.1716        4.9e-05   
   939/5000        -1240       0.0001163      0.2463        0.1713        4.9e-05   
   949/5000        -1251       0.000115       0.2921        0.1711        4.9e-05   
Time to complete logging: 0.3189

   959/5000        -1198       0.0001152      0.2367        0.1711        4.9e-05   
   969/5000        -3167       0.0001153      0.5868        0.1711        4.9e-05   
   979/5000        -2361       0.0001183      0.4604         0.171        4.9e-05 

  1739/5000        -2251       0.0001126      0.5831        0.1479       4.706e-05  
  1749/5000        -2702       0.0001173      0.6285        0.1477       4.706e-05  
Time to complete logging: 0.2968

  1759/5000        -3197        0.00011       0.6349        0.1475       4.706e-05  
  1769/5000        -2719       0.0001095      0.6282        0.1473       4.706e-05  
  1779/5000        -1633       0.0001062      0.4459         0.147       4.706e-05  
  1789/5000        -4714       0.0001131      0.9561        0.1466       4.706e-05  
  1799/5000        -3187       0.0001113      0.6321        0.1464       4.706e-05  
Time to complete logging: 0.277 

  1809/5000        -2918       0.0001211      0.6145        0.1462       4.706e-05  
  1819/5000        -3229       0.0001147      0.7524         0.146       4.706e-05  
  1829/5000        -2052       0.000113       0.4935        0.1458       4.706e-05  
  1839/5000        -2544       0.0001141      0.6835        0.1455       4.706e-05

  2599/5000        -3897       0.0001129      0.9936         0.128       4.52e-05   
Time to complete logging: 0.2883

  2609/5000        -3205       0.0001174      0.8505        0.1278       4.52e-05   
  2619/5000        -3226       0.0001096       0.895        0.1275       4.52e-05   
  2629/5000        -2678       0.0001094      0.6771        0.1271       4.52e-05   
  2639/5000        -2484       0.0001185      0.7383        0.1269       4.52e-05   
  2649/5000        -3113       0.0001138      0.8183        0.1266       4.52e-05   
Time to complete logging: 0.3311

  2659/5000        -3808       0.0001053      0.8992        0.1263       4.52e-05   
  2669/5000        -2946       0.0001157      0.7354        0.1259       4.52e-05   
  2679/5000        -2973       0.0001146      0.7525        0.1257       4.52e-05   
  2689/5000        -3513       0.0001198      0.8864        0.1254       4.52e-05   
  2699/5000        -2687       0.0001159      0.7614         0.125       4.52e-05 

Time to complete logging: 0.2573

  3459/5000        -2784       0.0001028      0.8001        0.09936      4.429e-05  
  3469/5000        -3012       0.0001036      0.9639        0.09909      4.429e-05  
  3479/5000        -2943       0.000107       0.9001        0.09873      4.429e-05  
  3489/5000        -3036       0.0001074      0.9821        0.09847      4.429e-05  
  3499/5000        -2528       0.0001074      0.8529        0.09816      4.429e-05  
Time to complete logging: 0.2631

  3509/5000        -2690       9.973e-05       0.889        0.09785      4.341e-05  
  3519/5000        -3321       0.000105       0.9704        0.09747      4.341e-05  
  3529/5000        -2708       0.0001073      0.9371        0.09709      4.341e-05  
  3539/5000        -2858       0.0001064      0.9369        0.09686      4.341e-05  
  3549/5000        -2906       0.0001076      0.9449        0.09664      4.341e-05  
Time to complete logging: 0.2308

  3559/5000        -2906       9.977e-05      0.

  4309/5000        -2456       0.000102       0.9293        0.08176      4.254e-05  
  4319/5000        -1921        0.00011       0.8986        0.08163      4.254e-05  
  4329/5000        -2159       0.0001075       0.907        0.0815       4.254e-05  
  4339/5000        -2908       0.0001316      0.9153        0.08142      4.254e-05  
  4349/5000        -2083       0.0001214      0.9449        0.0813       4.254e-05  
Time to complete logging: 0.2664

  4359/5000        -2321       0.0001567      0.9212        0.08112      4.254e-05  
  4369/5000        -2107       0.0001056      0.9073        0.08082      4.254e-05  
  4379/5000        -1907       0.0001424      0.9742        0.08057      4.254e-05  
  4389/5000        -2349       0.0001203      0.9565        0.08035      4.254e-05  
  4399/5000        -2164       0.0001228      0.9513        0.08032      4.254e-05  
Time to complete logging: 0.2813

  4409/5000        -2226       0.0001118      0.9434        0.08025      4.254e-05

In [8]:
run_steps = [100, 200, 500, 1000, 5000, 10000]

In [9]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:   0     time / step / sample / link: 0.03354
step:   1     time / step / sample / link: 3.837e-05
step:   2     time / step / sample / link: 3.575e-05
step:   3     time / step / sample / link: 3.492e-05
step:   4     time / step / sample / link: 3.769e-05
step:   5     time / step / sample / link: 3.504e-05
step:   6     time / step / sample / link: 3.607e-05
step:   7     time / step / sample / link: 3.398e-05
step:   8     time / step / sample / link: 4.167e-05
step:   9     time / step / sample / link: 3.674e-05
step:   10    time / step / sample / link: 3.619e-05
step:   11    time / step / sample / link: 3.608e-05
step:   12    time / step / sample / link: 3.723e-05
step:   13    time / step / sample / link: 3.728e-05
step:   14    time / step / sample / link: 3.846e-05
step:   15    time / step / sample / link: 3.816e-05
step:   16    time / step / sample / link: 3.833e-05
step:   17    time / step / sample / link: 3.557e-05

step:   51    time / step / sample / link: 4.067e-05
step:   52    time / step / sample / link: 3.816e-05
step:   53    time / step / sample / link: 3.793e-05
step:   54    time / step / sample / link: 3.734e-05
step:   55    time / step / sample / link: 3.856e-05
step:   56    time / step / sample / link: 3.781e-05
step:   57    time / step / sample / link: 4.806e-05
step:   58    time / step / sample / link: 3.961e-05
step:   59    time / step / sample / link: 3.452e-05
step:   60    time / step / sample / link: 3.726e-05
step:   61    time / step / sample / link: 3.672e-05
step:   62    time / step / sample / link: 3.858e-05
step:   63    time / step / sample / link: 3.939e-05
step:   64    time / step / sample / link: 3.994e-05
step:   65    time / step / sample / link: 3.981e-05
step:   66    time / step / sample / link: 3.887e-05
step:   67    time / step / sample / link: 3.88e-05
step:   68    time / step / sample / link: 3.446e-05
step:   69    time / step / sample / link: 3.74

step:   3     time / step / sample / link: 4.641e-05
step:   4     time / step / sample / link: 3.725e-05
step:   5     time / step / sample / link: 3.757e-05
step:   6     time / step / sample / link: 3.87e-05
step:   7     time / step / sample / link: 3.974e-05
step:   8     time / step / sample / link: 3.909e-05
step:   9     time / step / sample / link: 3.925e-05
step:   10    time / step / sample / link: 4.056e-05
step:   11    time / step / sample / link: 3.739e-05
step:   12    time / step / sample / link: 3.767e-05
step:   13    time / step / sample / link: 3.643e-05
step:   14    time / step / sample / link: 3.769e-05
step:   15    time / step / sample / link: 3.99e-05
step:   16    time / step / sample / link: 4.212e-05
step:   17    time / step / sample / link: 4.049e-05
step:   18    time / step / sample / link: 3.942e-05
step:   19    time / step / sample / link: 4.014e-05
step:   20    time / step / sample / link: 3.951e-05
step:   21    time / step / sample / link: 4.004

step:  159    time / step / sample / link: 4.078e-05
step:  160    time / step / sample / link: 3.875e-05
step:  161    time / step / sample / link: 3.867e-05
step:  162    time / step / sample / link: 3.842e-05
step:  163    time / step / sample / link: 3.692e-05
step:  164    time / step / sample / link: 3.889e-05
step:  165    time / step / sample / link: 3.758e-05
step:  166    time / step / sample / link: 3.545e-05
step:  167    time / step / sample / link: 3.913e-05
step:  168    time / step / sample / link: 3.718e-05
step:  169    time / step / sample / link: 3.84e-05
step:  170    time / step / sample / link: 3.867e-05
step:  171    time / step / sample / link: 3.972e-05
step:  172    time / step / sample / link: 3.895e-05
step:  173    time / step / sample / link: 4.516e-05
step:  174    time / step / sample / link: 3.973e-05
step:  175    time / step / sample / link: 3.668e-05
step:  176    time / step / sample / link: 3.778e-05
step:  177    time / step / sample / link: 3.93

step:  315    time / step / sample / link: 3.909e-05
step:  316    time / step / sample / link: 4.176e-05
step:  317    time / step / sample / link: 3.834e-05
step:  318    time / step / sample / link: 3.716e-05
step:  319    time / step / sample / link: 3.758e-05
step:  320    time / step / sample / link: 3.657e-05
step:  321    time / step / sample / link: 3.753e-05
step:  322    time / step / sample / link: 3.918e-05
step:  323    time / step / sample / link: 3.987e-05
step:  324    time / step / sample / link: 3.689e-05
step:  325    time / step / sample / link: 3.845e-05
step:  326    time / step / sample / link: 3.819e-05
step:  327    time / step / sample / link: 3.371e-05
step:  328    time / step / sample / link: 3.629e-05
step:  329    time / step / sample / link: 3.817e-05
step:  330    time / step / sample / link: 3.642e-05
step:  331    time / step / sample / link: 3.78e-05
step:  332    time / step / sample / link: 3.731e-05
step:  333    time / step / sample / link: 3.89

...

step:  9377   time / step / sample / link: 5.953e-05
step:  9378   time / step / sample / link: 4.041e-05
step:  9379   time / step / sample / link: 4.306e-05
step:  9380   time / step / sample / link: 4.117e-05
step:  9381   time / step / sample / link: 4.111e-05
step:  9382   time / step / sample / link: 3.641e-05
step:  9383   time / step / sample / link: 3.678e-05
step:  9384   time / step / sample / link: 3.357e-05
step:  9385   time / step / sample / link: 4.166e-05
step:  9386   time / step / sample / link: 3.933e-05
step:  9387   time / step / sample / link: 4.194e-05
step:  9388   time / step / sample / link: 3.882e-05
step:  9389   time / step / sample / link: 4.032e-05
step:  9390   time / step / sample / link: 3.796e-05
step:  9391   time / step / sample / link: 4.031e-05
step:  9392   time / step / sample / link: 4.045e-05
step:  9393   time / step / sample / link: 4.193e-05
step:  9394   time / step / sample / link: 3.922e-05
step:  9395   time / step / sample / link: 4.4

step:  9532   time / step / sample / link: 3.81e-05
step:  9533   time / step / sample / link: 3.853e-05
step:  9534   time / step / sample / link: 4.103e-05
step:  9535   time / step / sample / link: 6.83e-05
step:  9536   time / step / sample / link: 6.968e-05
step:  9537   time / step / sample / link: 4.279e-05
step:  9538   time / step / sample / link: 3.734e-05
step:  9539   time / step / sample / link: 3.632e-05
step:  9540   time / step / sample / link: 3.704e-05
step:  9541   time / step / sample / link: 3.669e-05
step:  9542   time / step / sample / link: 3.593e-05
step:  9543   time / step / sample / link: 3.605e-05
step:  9544   time / step / sample / link: 3.559e-05
step:  9545   time / step / sample / link: 3.619e-05
step:  9546   time / step / sample / link: 3.69e-05
step:  9547   time / step / sample / link: 3.565e-05
step:  9548   time / step / sample / link: 3.58e-05
step:  9549   time / step / sample / link: 3.578e-05
step:  9550   time / step / sample / link: 3.546e-

step:  9687   time / step / sample / link: 0.0001059
step:  9688   time / step / sample / link: 0.0001086
step:  9689   time / step / sample / link: 7.882e-05
step:  9690   time / step / sample / link: 6.687e-05
step:  9691   time / step / sample / link: 7.788e-05
step:  9692   time / step / sample / link: 6.555e-05
step:  9693   time / step / sample / link: 8.025e-05
step:  9694   time / step / sample / link: 8.079e-05
step:  9695   time / step / sample / link: 0.0001121
step:  9696   time / step / sample / link: 0.0001002
step:  9697   time / step / sample / link: 7.254e-05
step:  9698   time / step / sample / link: 6.644e-05
step:  9699   time / step / sample / link: 5.582e-05
step:  9700   time / step / sample / link: 5.08e-05
step:  9701   time / step / sample / link: 5.561e-05
step:  9702   time / step / sample / link: 4.475e-05
step:  9703   time / step / sample / link: 5.695e-05
step:  9704   time / step / sample / link: 4.848e-05
step:  9705   time / step / sample / link: 4.69

step:  9842   time / step / sample / link: 3.892e-05
step:  9843   time / step / sample / link: 3.84e-05
step:  9844   time / step / sample / link: 3.97e-05
step:  9845   time / step / sample / link: 3.801e-05
step:  9846   time / step / sample / link: 3.842e-05
step:  9847   time / step / sample / link: 3.461e-05
step:  9848   time / step / sample / link: 3.826e-05
step:  9849   time / step / sample / link: 3.829e-05
step:  9850   time / step / sample / link: 3.983e-05
step:  9851   time / step / sample / link: 3.672e-05
step:  9852   time / step / sample / link: 4.055e-05
step:  9853   time / step / sample / link: 3.686e-05
step:  9854   time / step / sample / link: 3.659e-05
step:  9855   time / step / sample / link: 3.79e-05
step:  9856   time / step / sample / link: 4.013e-05
step:  9857   time / step / sample / link: 3.788e-05
step:  9858   time / step / sample / link: 3.896e-05
step:  9859   time / step / sample / link: 3.74e-05
step:  9860   time / step / sample / link: 3.833e-

step:  9997   time / step / sample / link: 3.295e-05
step:  9998   time / step / sample / link: 3.487e-05
step:  9999   time / step / sample / link: 3.553e-05
done.
Samples saved to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_38/samples_history/samples_history_10000.pkl.
--------------------------------------------------------------------------------



In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)